# FirecREST Tutorial - Part 1: HTTP Requests

The point of the tutorial is to learn how to interact with the FirecREST’s API through a typical workflow.

## Overview of the tutorial

1. Learn how to make http requests in python, by testing the most common ones: `GET` and `POST`
- Upload the input file to the cluster's filesystem
- Run a simple job: the job will be a simple SHA-1 calculation of the file we uploaded
- Learn how to transfer files through the `Storage Microservice`
- Download and verify the results

To run a jupyter cell click on it and then press CTRL+Enter. Let's try the first one to make sure we have the necessary packages.

#### Cell 1A

In [ ]:
import json
import requests
import time

#### Cell 1B

In [ ]:
# If that doesn't work you may need to install requests package:
!pip install --user requests

### HTTP requests basics

Before starting using the API it's important we are all a little familiar with HTTP requests and how they are used.

FirecREST API is based on REST principles: data resources are accessed via standard HTTP requests to an API endpoint.

Every request is made of:

- the endpoint or requested URL
- the method (one of GET, POST, PUT and DELETE depending on the appropriate action)
- the headers (metadata necessary for the request)
- the body (form data, files to be uploaded, etc)

The necessary information for every call is passed through query parameters, the headers and the body of the request. You can find all the available API calls of FirecREST in the reference section and here is a quick overview of the methods:

Method | Description
:---: | :---
GET | Used for retrieving resources.
POST | Used for creating/updating resources.
PUT | Used for creating/updating resources.*
DELETE | Used for deleting resources.

\* The difference between POST and PUT is that PUT requests are idempotent. That is, calling the same PUT request multiple times will always produce the same result. In contrast, calling a POST request repeatedly have side effects of creating the same resource multiple times.

Similar to the requests, the response of FirecREST will consist of:

- a status code
- the headers
- the body in json form

Here is a quick overview of the status codes and their meaning.

\# | Category | Description
:---: | :--- | :---
1xx | Informational | Communicates transfer protocol-level information.
2xx | Success | Indicates that the client’s request was accepted successfully.
3xx | Redirection | Indicates that the client must take some additional action in order to complete their request.
4xx | Client Error | This category of error status codes points the finger at clients.
5xx | Server Error | The server takes responsibility for these error status codes.

## Testbuild

In this tutorial are not going to access the resources of CSCS but a fake cluster (named `cluster`). In this cluster slurm is used for the job submission. Your username and your token should have been sent to you already!

So let's setup the credentials, as well as the FirecREST IP, as we will use it many times during the tutorial.

#### Cell 1C

In [ ]:
TOKEN='<token>'
FIRECREST_IP = 'http://148.187.98.88:8000'

## Test the credentials with a simple call

#### Cell 1D

In [ ]:
response = requests.get(
    url=f'{FIRECREST_IP}/status/systems',
    headers={'Authorization': f'Bearer {TOKEN}'}
)

print(response.status_code)
print(response.headers)
print(response.json())

We can see the three parts of the response:

1. the status code in `response.status_code`
2. the headers in `response.headers`
3. the json part by calling `response.json()`

But the output is not very readable, so now we are going to define a function that will print the response in a more readable way.

Run the next cell to get the output in better format.

#### Cell 1E

In [ ]:
# This function is useful only to print the response in a nicer way
def handle_response(response):
    print("\nResponse status code:")
    print(response.status_code)
    print("\nResponse headers:")
    print(json.dumps(dict(response.headers), indent=4))
    print("\nResponse json:")
    try:
        print(json.dumps(response.json(), indent=4))
    except json.JSONDecodeError:
        print("-")

handle_response(response)

## List the contents of a directory

We will start with a call that returns the contents of a directory in the filesystem.

[Link to FirecREST call](http://148.187.97.201:8000/#/Utilities/get_utilities_ls)

Things to notice:

1. This is a `GET` request
- The endpoint is `/utilities/ls`
- Set the required fields
  - `targetPath`, which will be the location in the machine's filesystem
  - `X-Machine-Name`, the filesystem we are interested in
  - the `Authorization` token

Replace `targetPath` with your user's home and run the cell.
<a id='ls-cell'></a>
#### Cell 1F

In [ ]:
targetPath = '/home/llama'
machine = 'cluster'

response = requests.get(
    url=f'{FIRECREST_IP}/utilities/ls',
    headers={'Authorization': f'Bearer {TOKEN}',
             'X-Machine-Name': machine},
    params={'targetPath': f'{targetPath}'}
)

handle_response(response)

### Exercise 1: Finding error messages on the response in case of invalid requests

1. Change `targetPath` to an invalid path, run it and find the corresponding error message in the response.
2. Change `targetPath` to another's user home ( `llama` ), where you don't have access, run it and find the corresponding error message in the response.

Run the next cell to see the solutions.

#### Cell 1G

In [ ]:
%cat solutions/ls_errors.txt

### cURL equivalent

```bash
curl -X GET "${FIRECREST_IP}/utilities/ls?targetPath=/home/llama" -H "Authorization: Bearer ${TOKEN}" -H "X-Machine-Name: cluster"
```

## Upload a small file with the blocking call

The second type of calls we are going to look at today is a `POST` request. For this purpose we are going to upload a small file (<5MB) to our cluster. For this exercise you can choose your favorite file or just run the next cell to create one.

#### Cell 1H

In [ ]:
%%writefile files/firecrest_input_file.txt
Hello!

Let's first have a look at the request in this [link](http://148.187.97.201:8000/#/Utilities/post_utilities_upload).

We will only set the required arguments: `targetPath`, `X-Machine-Name`, `file` and `Authorization`. Notice that `targetPath` is a **form-data** parameter and not a **query** parameter as before, so it is passed as a argument in the `data` dictionary.

In order to pass the `file` argument, we don't just give the name of the file as a string; we pass a file object in binary form. For this reason we will open the file with the flags `'rb'`.

#### Cell 1I

In [ ]:
targetPath = '/home/llama'
machine = 'cluster'
localPath = 'files/firecrest_input_file.txt'

response = requests.post(
    url=f'{FIRECREST_IP}/utilities/upload',
    headers={'Authorization': f'Bearer {TOKEN}',
             'X-Machine-Name': machine},
    data={'targetPath': targetPath},
    files={'file': open(localPath, 'rb')}
)

handle_response(response)

### cURL equivalent

```bash
curl -X POST "${FIRECREST_IP}/utilities/upload" \
     -H "Authorization: Bearer ${TOKEN}" \
     -H "X-Machine-Name: cluster" \
     -F "targetPath=/home/llama" \
     -F "file=@files/firecrest_input_file.txt"
```

### Exercise:

If you want to make sure you have uploaded your file successfully you can run the [call](#ls-cell) we saw in the previous section.